# Exceptions and File Handling

- Understanding Exceptions
- Handling Exceptions
- Defining Custom Exceptions
- Reading/Writing Files
- Navigating the File System

# Understanding Exceptions

Things do not always go as planned. If you don't want your program to just crash you'll need to be able to handle exceptions.

# Same Code but Different Style

You have a couple options.

In some cases you can do some preliminary checks to make sure that your code will successfully execute.

Or you can dive in an catch errors as they happen.

# Look Before You Leap

In [ ]:
c = 200
if a != 0:
    c = 200 / a

# Easier to Ask Forgiveness than Permission

In [ ]:
c = 200
try:
    c = 200 / a
except ZeroDivisionError:
    pass

# Catching Exceptions

The `try` keyword starts the block where exceptions are to be handled. The `except` keyword denotes which exception classes are handled.

You can also define an `else` block which will only excute if no exceptions were raised. A `finally` block is also optional and will be excuted regardless of whether there were exceptions or not.

In [ ]:
try:
    # Something dangerous
except IndexError:
    # Handle the error
else:
    # No problems so do something
finally:
    # Clean up either way

# Catching Multiple Types of Exceptions

You can handle multiple types of exceptions in one `except` block.

In [ ]:
try:
    # Something dangerous
except (TypeError, IndexError):
    # Handle either type

# Handling Different Exceptions Differently

You can also define multiple `except` cases to handle different exception types in different ways.

In [ ]:
try:
    # Something dangerous
except TypeError:
    # Handle type error
except IndexError:
    # Handle index error
except:
    # Handle all other types

# Getting Exception Info

In [2]:
try:
    # Something dangerous
    'a' + 1
except TypeError as e:
    print(e)
    print(e.args)
    # Handle type error

must be str, not int
('must be str, not int',)


# Built-in Exceptions

Some common exception classes:

- `Exception` - Base exception class
- `AttributeError` - Attempted to access an object attribute that doesn't exist
- `IOError` - I/O related error (file not found, disk full, etc)
- `ImportError` - Module import error
- `IndexError` - Accessing index outside of list
- `KeyError` - Accessing dictionary key that doesn't exist

For a full list see http://docs.python.org/library/exceptions.html

# Creating Exceptions

Creating exceptions is as easy as creating a class.

In [ ]:
class EveryonePanicException(Exception):
    pass

# Creating Exceptions Expanded

As with any class you can also pass additional information into your exceptions.

In [5]:
class EveryonePanicException(Exception):
    
    def __init__(self, reason):
        self.reason = reason

    def __str__(self):
        return 'Everyone panic! %s' % self.reason

# Raising Exceptions

The `raise` keyword is used to raise the specified exception.

In [ ]:
raise EveryonePanicException("It's Godzilla!")

# Raising Exceptions Again

If you've caught an exception that you don't intend to handle then you can re-raise the last exception with raise.

In [7]:
try:
    raise EveryonePanicException("It's Godzilla!")
except EveryonePanicException:
    print("There was an exception.")
    raise

There was an exception.


EveryonePanicException: Everyone panic! It's Godzilla!

# Supporting Different Versions

`lxml` is a high-performance XML parser which supports the same API as the XML parser in the standard library. You can fallback to the standard libary version if it isn't installed/available.

In [9]:
try:
  from lxml import etree
except ImportError:
  try:
    import xml.etree.cElementTree as etree
  except ImportError:
    import xml.etree.ElementTree as etree
    
etree.__file__

'/usr/lib/python3.6/xml/etree/cElementTree.py'

# Opening Files

The built-in `open` function is used to open files. It takes the filename, mode (optional), and buffer size (optional). This is implemented as stdio `fopen()` in the underlying C. The mode defaults to 'r' (for reading).

In [11]:
open_file = open('MA792-002-Python-4.ipynb')
contents = open_file.readlines() # Reads entire file
open_file.close()

In [ ]:
# The File Object

Let's take a look at what methods are on the `file` type.

In [13]:
dir(file)

NameError: name 'file' is not defined

# Opening Files Safely

You can avoid having to remember to close the file by opening the file using a `with` statement:

In [ ]:
with open('example.txt') as f:
    # do something with f
    print f.readlines()